# Segmenting and Clustering Neighborhoods in Toronto

### Import required libraries

In [ ]:
#!pip install beautifulsoup4
#!pip install lxml

from bs4 import BeautifulSoup
import requests
import pandas as pd

## Get Canadian Postal Codes (M) from Wikipedia

### In a separate notebook, we parsed using Beautiful Soup

We did lots of clean-up and saved the result to a csv-file named 'Postal.csv'  
Now, we're going to pick-it-up, so we can continue with the task of getting the geo-data  
and combining into a single DataFrame.

In [ ]:
df_zip = pd.read_csv('Postal.csv')
df_zip.head()

In [ ]:
print(df_zip.shape)

## Get geospatial coordinates of zip codes and combine into a DataFrame

### This is how we would lookup geospatial coordinates using the Geocoder Python library

In [ ]:
!pip install geocoder

In [ ]:
import geocoder # import geocoder

In [ ]:
for index in df_zip.index:
    Postcode = df_zip['Postcode'].iloc[index]
    Borough = df_zip['Borough'].iloc[index]
    Neighbourhood = df_zip['Neighbourhood'].iloc[index]
    
    # initialize your variable to None
    lat_lng_coords = None
    latitude = 0
    longitude = 0

    count = 0
    # loop until you get the coordinates
    if False:   #The while loop never came back, so I abandoned Geocoder
        while((lat_lng_coords is None) | (count >9) ):
            g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
            lat_lng_coords = g.latlng
            count =+ 1

        if lat_lng_coords is None:
            latitude = 0
            longitude = 0
        else:
            latitude = lat_lng_coords[0]
            longitude = lat_lng_coords[1]

#   print(Postcode, Borough, Neighbourhood, latitude, longitude)

### And this is how we actually get the geospatial data

In [ ]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

In [ ]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.rename(columns={'Postal Code':'Postcode'}, inplace=True)

Before we set the indexes, look at the shapes

In [ ]:
print('Before setting Index')
print('    df_zip:',df_zip.shape)
print('    df_geo:',df_geo.shape)

It is helpful to use the index for each DataFrame

In [ ]:
df_geo.set_index('Postcode', inplace=True)
df_zip.set_index('Postcode', inplace=True)
print('df_zip:',df_zip.shape)
print('df_geo:',df_geo.shape)

In [ ]:
df_geo.head()

### Now, Join the two DataFrames, adding new columns, combining by matching Index

In [ ]:
df_combined = df_zip.join(df_geo)
print(df_combined.shape)

In [ ]:
# we save it with index=Postcode
df_combined.to_csv('Toronto.csv')

In [ ]:
df_combined.reset_index(inplace=True)

In [ ]:
df_combined

In [ ]:
df_combined.shape